In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import os
import time
import imgaug as ia
from imgaug import augmenters as iaa
# image augmentation
import cv2
from PIL import Image
from torchvision import transforms
import xml.etree.ElementTree as ET
# xml 읽어 오는 라이브러리
from xml.etree.ElementTree import Element, ElementTree

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
def makeBox(voc_im,bbox,objects):
    image = voc_im.copy()
    for i in range(len(objects)):                 # 이미지 내에 box 그리는 함수
        cv2.rectangle(image,(int(bbox[i][0]),int(bbox[i][1])),(int(bbox[i][2]),int(bbox[i][3])),color = (0,255,0),thickness = 1)
        cv2.putText(image, objects[i], (int(bbox[i][0]), int(bbox[i][1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
    return image

In [ ]:
import os

annotation_train_path = '/content/drive/MyDrive/YAI_Toy_Project/data/annotation_train'
image_train_path = '/content/drive/MyDrive/YAI_Toy_Project/data/image_train/'

In [ ]:
def list_files(directory):
    files= []

    for dirpath, dirnames, filenames in os.walk(directory):
        if filenames:
            for f in filenames:
                files.append(f)

    return files

In [ ]:
def list_json_files(directory):
    files= []

    for dirpath, dirnames, filenames in os.walk(directory):
        if filenames:
            for f in filenames:
                files.append(os.path.join(dirpath, f))

    return files

In [ ]:
annotation_list = list_files(annotation_train_path)
image_list = list_files(image_train_path)

# 파일 목록 정렬
annotation_list.sort()
image_list.sort()

In [ ]:
# 파일 목록 출력
print("annotation in directory:", annotation_list)

annotation in directory: ['나무_10_남_00013.json', '나무_10_남_00022.json', '나무_10_남_00023.json', '나무_10_남_00040.json', '나무_10_남_00051.json', '나무_10_남_00064.json', '나무_10_남_00065.json', '나무_10_남_00124.json', '나무_10_남_00148.json', '나무_10_남_00175.json', '나무_10_남_00182.json', '나무_10_남_00188.json', '나무_10_남_00192.json', '나무_10_남_00198.json', '나무_10_남_00204.json', '나무_10_남_00207.json', '나무_10_남_00224.json', '나무_10_남_00241.json', '나무_10_남_00269.json', '나무_10_남_00273.json', '나무_10_남_00275.json', '나무_10_남_00279.json', '나무_10_남_00311.json', '나무_10_남_00314.json', '나무_10_남_00332.json', '나무_10_남_00343.json', '나무_10_남_00353.json', '나무_10_남_00374.json', '나무_10_남_00377.json', '나무_10_남_00379.json', '나무_10_남_00380.json', '나무_10_남_00427.json', '나무_10_남_00431.json', '나무_10_남_00434.json', '나무_10_남_00446.json', '나무_10_남_00451.json', '나무_10_남_00452.json', '나무_10_남

In [ ]:
print("image in directory:", image_list)

image in directory: ['나무_10_남_00013.jpg', '나무_10_남_00022.jpg', '나무_10_남_00023.jpg', '나무_10_남_00040.jpg', '나무_10_남_00051.jpg', '나무_10_남_00064.jpg', '나무_10_남_00065.jpg', '나무_10_남_00124.jpg', '나무_10_남_00148.jpg', '나무_10_남_00175.jpg', '나무_10_남_00182.jpg', '나무_10_남_00188.jpg', '나무_10_남_00192.jpg', '나무_10_남_00198.jpg', '나무_10_남_00204.jpg', '나무_10_남_00207.jpg', '나무_10_남_00224.jpg', '나무_10_남_00241.jpg', '나무_10_남_00269.jpg', '나무_10_남_00273.jpg', '나무_10_남_00275.jpg', '나무_10_남_00279.jpg', '나무_10_남_00311.jpg', '나무_10_남_00314.jpg', '나무_10_남_00332.jpg', '나무_10_남_00343.jpg', '나무_10_남_00353.jpg', '나무_10_남_00374.jpg', '나무_10_남_00377.jpg', '나무_10_남_00379.jpg', '나무_10_남_00380.jpg', '나무_10_남_00427.jpg', '나무_10_남_00431.jpg', '나무_10_남_00434.jpg', '나무_10_남_00446.jpg', '나무_10_남_00451.jpg', '나무_10_남_00452.jpg', '나무_10_남_00467.jpg', '나무_10_남_00473.jpg', '나ᄆ

In [ ]:
import json

# 유효한 좌표를 가진 바운딩 박스만 추출하는 함수
def filter_valid_bboxes(bboxes):
    valid_bboxes = []
    for bbox in bboxes:
        x, y, w, h = bbox
        # 바운딩 박스의 너비와 높이가 양수인지 확인하여 유효한 바운딩 박스인지 판단
        if w > 0 and h > 0:
            bbox = x, y, x+w, y+h
            valid_bboxes.append(bbox)
    return valid_bboxes


# JSON 파일을 파싱하여 필요한 정보를 추출하는 함수
def parse_json(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)

    file_name = data['meta']['img_id']
    object_names = [bbox['label'] for bbox in data['annotations']['bbox']]
    bboxes = [(bbox['x'], bbox['y'], bbox['w'], bbox['h']) for bbox in data['annotations']['bbox']]

    # 유효한 좌표를 가진 바운딩 박스만 추출
    valid_bboxes = filter_valid_bboxes(bboxes)

    return file_name, object_names, valid_bboxes


# 디렉토리 내의 모든 JSON 파일 목록 가져오기
json_files = list_json_files(annotation_train_path)
# 추출된 레이블을 담을 집합 초기화
label_set = set()
count= 0
# 각 JSON 파일을 파싱하고 레이블 집합에 추가
for json_file in json_files:
    _, object_names, _ = parse_json(json_file)
    count+=1
    if count%11200 ==0: print('TL done')

    for name in object_names:
        label_set.add(name)

label_set = sorted(list(label_set))
label_dic = {}

for i, key in enumerate(label_set):
    label_dic[key] = i

# 결과 출력
print("label dic : ",label_dic)
print("Extracted Labels:", label_set)

TL done


In [ ]:
image_files= list_json_files(image_train_path)
json_files= list_json_files(annotation_train_path)

In [ ]:
class Pascal_Vo(Dataset):
    def __init__(self, image_list, annotation_list, len_data):
        self.image_list = image_list
        self.annotation_list = annotation_list
        self.len_data = len_data                 # 데이터셋 길이
        self.to_tensor = transforms.ToTensor()
        self.flip = iaa.Fliplr(0.5)              # augmentation flip 사용
        self.resize = iaa.Resize({'shorter-side': 600, "longer-side":"keep-aspect-ratio"})   # augmentation resize 사용

    def __len__(self):
        return self.len_data

    def __getitem__(self, idx):
        image_name = self.image_list[idx]
        annotation_name = self.annotation_list[idx]

        image_path = image_files[idx]
        annotation_path = json_files[idx]
        file_name, object_name, bbox = parse_json(annotation_path)
        image = Image.open(image_path).convert('RGB')
        image = np.array(image)

        image, bbox = self.flip(image=image, bounding_boxes=np.array([bbox]))
        image, bbox = self.resize(image=image, bounding_boxes=bbox)
        bbox = bbox.squeeze(0).tolist()
        image = self.to_tensor(image)


        # Check if any bounding box has invalid dimensions
        invalid_bbox = any(w <= 0 or h <= 0 for x, y, w, h in bbox)

        if invalid_bbox:
            # Skip this sample and return None
            return None, None
        else:
            # Prepare targets
            targets = []
            d = {}
            d['boxes'] = torch.tensor(bbox)
            d['labels'] = torch.tensor([label_dic[x] for x in object_name], dtype=torch.int64)
            targets.append(d)

            return image, targets

In [ ]:
len(label_set)

In [ ]:
backbone = torchvision.models.vgg16(pretrained=True).features[:-1]    # pretrained model 불러오기
backbone_out = 512              # output size
backbone.out_channels = backbone_out

anchor_generator = torchvision.models.detection.rpn.AnchorGenerator(sizes=((128,256,512),),aspect_ratios=((0.5,1.0,2.0),))
# detection generator 만들기

resolution = 7
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],output_size=resolution,sampling_ratio=2)    # ROi 정의

box_head = torchvision.models.detection.faster_rcnn.TwoMLPHead(in_channels = backbone_out*(resolution**2),representation_size=4096)
box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(4096,47) # class 의 개수

# faster rcnn model Linear 부분 + Predictor 부분


model = torchvision.models.detection.FasterRCNN(backbone, num_classes=None,
                        min_size= 600, max_size = 1000,
                        rpn_anchor_generator= anchor_generator,
                        rpn_pre_nms_top_n_train=6000, rpn_pre_nms_top_n_test=6000,
                        rpn_post_nms_top_n_train= 2000, rpn_post_nms_top_n_test=300,
                        rpn_nms_thresh=0.7, rpn_fg_iou_thresh=0.7, rpn_bg_iou_thresh=0.3,
                        rpn_batch_size_per_image= 256, rpn_positive_fraction=0.5,
                        box_roi_pool=roi_pooler, box_head= box_head, box_predictor = box_predictor,
                        box_score_thresh=0.05, box_nms_thresh=0.7, box_detections_per_img=300,
                        box_fg_iou_thresh=0.5, box_be_iou_thresh=0.5,
                        box_batch_size_per_image=128, box_positive_fraction=0.25
                    )

# model 하이퍼 파리미터 부분 : output  size 바꾸면 다 바꿔야 됨

for param in model.rpn.parameters():
    torch.nn.init.normal_(param, mean=0.0, std=0.01)     # normalize

for name, param in model.roi_heads.named_parameters():
    if "bbox_pred" in name:
        torch.nn.init.normal_(param, mean=0.0, std=0.001)
    elif "weight" in name:
        torch.nn.init.normal_(param, mean =0.0, std=0.01)
    if "bias" in name:
        torch.nn.init.zeros_(param)

In [ ]:
def Total_loss(loss):
    loss_objectness = loss['loss_objectness']
    loss_rpn_box_reg = loss['loss_rpn_box_reg']
    loss_classifier = loss['loss_classifier']
    loss_box_reg = loss['loss_box_reg']

    rpn_total = loss_objectness + 10*loss_rpn_box_reg  #(람다라고 보면된다)
    fast_rcnn_total = loss_classifier + 1*loss_box_reg

    total_loss = rpn_total + fast_rcnn_total

    return total_loss

In [ ]:
total_epoch = 15

len_data = 192 # 한 에폭당 이미지 개수

loss_sum = 0

optimizer = torch.optim.SGD(params = model.parameters(), lr=0.001, momentum=0.9, weight_decay = 0.0005) # SGD optimizer
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, total_epoch,eta_min=0.00001)  # scheduler 사용

start_epoch = 0
start_idx = 0

print("start_epoch = {} , start_idx = {}".format(start_epoch,start_idx))

print("Training Start")


model = model.to(device)
model.train()       # train 모드
start = time.time()

for epoch in range(start_epoch, total_epoch):
    dataset = Pascal_Vo(image_list, annotation_list, len_data - start_idx)   # dataset 불러오기
    dataloader = DataLoader(dataset, shuffle=True)    # dataloader 생성


    # for i, (image, targets) in enumerate(dataloader, start_idx):   # index image target(label) 불러오기
    for i, (image, targets) in enumerate(dataloader, start_idx):
        image = image.to(device)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()
        # print("i : ", i)
        # print("image : ", image)
        # print("targets : ", targets)

        targets[0]['boxes'].squeeze_(0)
        targets[0]['labels'].squeeze_(0)
        # print("targets : ", targets)
        loss = model(image,targets)
        total_loss = Total_loss(loss)
        loss_sum += total_loss


        total_loss.backward()
        optimizer.step()

    start_idx = 0
    scheduler.step()

    state ={
      'epoch' : epoch,
      'iter' :i+1,
      'state_dict' :model.state_dict(),
      'optimizer':optimizer.state_dict(),
      'scheduler' : scheduler.state_dict()
    }
    print('epoch:' + str(epoch))
    print('loss:' + str(total_loss))

In [ ]:
from torchvision.ops import nms

model.eval()
image = 0
for i, (image_d, targets) in enumerate(dataloader, 13):
  image = image_d
  break

image = image.to(device)
preds = model(image)


boxes = preds[0]['boxes']
labels = preds[0]['labels']
scores = preds[0]['scores']  # 신뢰도 점수

# NMS 적용
nms_indices = nms(boxes, scores, iou_threshold=0.1)

nms_boxes = boxes[nms_indices]
nms_labels = labels[nms_indices].tolist()



objects = []
for lb in nms_labels:
    objects.append([k for k, v in label_dic.items() if v == lb][0])

plot_image = image.squeeze().permute(1,2,0).cpu().numpy()
answer = makeBox(plot_image, nms_boxes, objects)

# 한국어 폰트 설정
# plt.rcParams['font.family'] = 'NanumGothic'

plt.imshow(answer)

In [ ]:
# 모델의 state_dict 저장
torch.save(model.state_dict(), 'model.pth')

# 옵티마이저의 state_dict 저장 (필요한 경우)
torch.save(optimizer.state_dict(), 'optimizer.pth')
